In [11]:
# NOTEARS → Edge Sweep(k=1..Kmax) → F(m/mw/both) → O/F/OF × 10-fold CV
# Models: Logit, RF, XGBoost(있으면), LightGBM(있으면)
# Outputs: logs/run_log.csv, reports/report_long.csv, reports/report_pivot.csv, reports/report_all_like_user.csv, reports/best_per_model.csv

import warnings, time
from pathlib import Path
from typing import List, Dict

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, brier_score_loss
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings("ignore")

# optional
try:
    from lightgbm import LGBMClassifier
    _HAS_LGBM = True
except Exception:
    _HAS_LGBM = False

try:
    from xgboost import XGBClassifier
    _HAS_XGB = True
except Exception:
    _HAS_XGB = False

# gCastle NOTEARS
try:
    from castle.algorithms import Notears
    _HAS_NOTEARS = True
except Exception as e:
    _HAS_NOTEARS = False
    raise ImportError("gCastle의 NOTEARS 필요: pip install gcastle torch") from e


In [12]:
# 기본 설정
DATA_PATH = "./training_data.csv"
LABEL_COL = "label"
ID_PREFIX = "Unnamed"
N_SPLITS = 10
RANDOM_STATE = 42
DAG_NAME = "NOTEARS"
CLASS_WEIGHT_BALANCED = True
VERBOSE_PRINT = True  # 학습 1회당 요약 1줄

# 모델 파라미터
LGBM_PARAMS = dict(
    n_estimators=300, learning_rate=0.05, num_leaves=63, max_depth=-1,
    subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0, reg_alpha=0.0,
    objective="binary", class_weight="balanced", random_state=RANDOM_STATE, n_jobs=-1
)

XGB_PARAMS = dict(
    n_estimators=300, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8,
    reg_lambda=1.0, reg_alpha=0.0, objective="binary:logistic", eval_metric="auc",
    random_state=RANDOM_STATE, n_jobs=-1
)


In [13]:
# 유틸/평가/모델팩토리/저장

def expected_calibration_error(y_true, y_prob, n_bins=10):
    y_true = np.asarray(y_true); y_prob = np.asarray(y_prob)
    bins = np.linspace(0,1,n_bins+1); ece=0.0; N=len(y_true)
    for b in range(n_bins):
        L,R = bins[b], bins[b+1]
        m = (y_prob>=L)&(y_prob<(R if b<n_bins-1 else R+1e-12))
        if not np.any(m): continue
        ece += (m.sum()/N)*abs(y_true[m].mean()-y_prob[m].mean())
    return float(ece)

def run_cv(model, X, y, n_splits=10, seed=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    f1s, auprcs, aurocs, briers, eces = [], [], [], [], []
    Xv = X.values.astype(np.float32); yv = y.values.astype(int)
    for tr, te in skf.split(Xv, yv):
        X_tr, X_te, y_tr, y_te = Xv[tr], Xv[te], yv[tr], yv[te]
        is_lgbm = (model.__class__.__name__ == "LGBMClassifier")
        if is_lgbm:
            X_tr2, X_val, y_tr2, y_val = train_test_split(X_tr, y_tr, test_size=0.2, random_state=seed, stratify=y_tr)
            model.fit(X_tr2, y_tr2, eval_set=[(X_val, y_val)], eval_metric="auc")
        else:
            model.fit(X_tr, y_tr)
        proba = model.predict_proba(X_te)[:,1] if hasattr(model,"predict_proba") else model.predict(X_te).astype(float)
        pred = (proba>=0.5).astype(int)
        f1s.append(f1_score(y_te, pred, zero_division=0))
        auprcs.append(average_precision_score(y_te, proba) if len(np.unique(y_te))>1 else np.nan)
        aurocs.append(roc_auc_score(y_te, proba) if len(np.unique(y_te))>1 else np.nan)
        briers.append(brier_score_loss(y_te, proba))
        eces.append(expected_calibration_error(y_te, proba, 10))
    return dict(f1=float(np.nanmean(f1s)), AUPRC=float(np.nanmean(auprcs)), AUROC=float(np.nanmean(aurocs)),
                Brier=float(np.nanmean(briers)), ECE=float(np.nanmean(eces)))

def pick_top_k_edges(W, top_k):
    n=W.shape[0]; flat=[]
    for i in range(n):
        for j in range(n):
            if i==j: continue
            w=W[i,j]
            if w!=0.0: flat.append((i,j,w))  # i<-j
    flat.sort(key=lambda t:(abs(t[2]), t[2]), reverse=True)
    return flat[:top_k]

def build_feature_df(X_base, cols, W, feature_type, top_k):
    edges=pick_top_k_edges(W, top_k); feats={}
    for i,j,w in edges:
        A,B = cols[j], cols[i]
        if feature_type=="mul":
            feats[f"{A}_mul_{B}"] = (X_base[A]*X_base[B]).values
        elif feature_type=="mulw":
            feats[f"{A}_mulw_{B}"] = (w*(X_base[A]*X_base[B])).values
        elif feature_type=="both":
            feats[f"{A}_mul_{B}"]   = (X_base[A]*X_base[B]).values
            feats[f"{A}_mulw_{B}"]  = (w*(X_base[A]*X_base[B])).values
    return pd.DataFrame(feats, index=X_base.index)

def make_models():
    ms=[("Logit", LogisticRegression(solver="lbfgs", max_iter=200,
                                     class_weight=("balanced" if CLASS_WEIGHT_BALANCED else None))),
        ("RandomForest", RandomForestClassifier(n_estimators=400,
                                               class_weight=("balanced" if CLASS_WEIGHT_BALANCED else None),
                                               n_jobs=-1, random_state=RANDOM_STATE))]
    if _HAS_XGB:
        ms.append(("XGBoost", XGBClassifier(**XGB_PARAMS, verbosity=0)))
    if _HAS_LGBM:
        ms.append(("LightGBM", LGBMClassifier(**LGBM_PARAMS)))
    return ms

def ensure_dirs():
    Path("./logs").mkdir(parents=True, exist_ok=True)
    Path("./reports").mkdir(parents=True, exist_ok=True)

def append_log_csv(row: Dict):
    ensure_dirs()
    p=Path("./logs/run_log.csv")
    pd.DataFrame([row]).to_csv(p, mode="a", index=False, encoding="utf-8-sig", header=not p.exists())

def one_line(dag,k,model,set_name,met,secs):
    return (f"DAG={dag} | k={k} | model={model} | set={set_name} | "
            f"f1={met['f1']:.4f}, AUPRC={met['AUPRC']:.4f}, AUROC={met['AUROC']:.4f}, "
            f"Brier={met['Brier']:.4f}, ECE={met['ECE']:.4f} | time={secs:.2f}s")

def save_user_style_wide_csv(all_rows: List[Dict], out_path: Path):
    groups=["O","F","OF","F-O","OF-O"]; mets=["f1","AUPRC","AUROC","Brier","ECE"]
    header=dict(DAG="DAG",Model="Model",Feature="Feature")
    for g in groups:
        for m in mets: header[f"{g}:{m}"]=m
    rows=[header]; key_map={}
    for r in all_rows:
        if r["set"] not in {"O","F","OF"}: continue
        key=(r["dag"], r["model"], r["feature"], r["k"])
        if key not in key_map:
            base=dict(DAG=r["dag"],Model=r["model"],Feature=r["feature"],k=r["k"])
            for g in groups:
                for m in mets: base[f"{g}:{m}"]=np.nan
            key_map[key]=base
        for m in mets: key_map[key][f"{r['set']}:{m}"]=r[m]
    for rec in key_map.values():
        for m in mets:
            if pd.notna(rec[f"F:{m}"]) and pd.notna(rec[f"O:{m}"]):
                rec[f"F-O:{m}"]=rec[f"F:{m}"]-rec[f"O:{m}"]
            if pd.notna(rec[f"OF:{m}"]) and pd.notna(rec[f"O:{m}"]):
                rec[f"OF-O:{m}"]=rec[f"OF:{m}"]-rec[f"O:{m}"]
        rows.append(rec)
    wide=pd.DataFrame(rows)
    order=["DAG","Model","Feature"]+[f"{g}:{m}" for g in groups for m in mets]+["k"]
    wide=wide[order]
    out_path.parent.mkdir(parents=True, exist_ok=True)
    wide.to_csv(out_path, index=False, encoding="utf-8-sig")


In [14]:
# 데이터 로드 → NOTEARS(linear) → A(i<-j) → W(표준화 OLS)
ensure_dirs()
path = Path(DATA_PATH); assert path.exists(), f"not found: {path}"
df = pd.read_csv(path); assert LABEL_COL in df.columns, f"'{LABEL_COL}' not found"
id_cols = [c for c in df.columns if c.startswith(ID_PREFIX)]
feature_cols = [c for c in df.columns if c not in id_cols + [LABEL_COL]]
X_base = df[feature_cols].copy()
y = df[LABEL_COL].astype(int).copy()

# 분산0 제거
var0 = [c for c in feature_cols if np.isclose(X_base[c].var(ddof=0), 0.0)]
cols_used = [c for c in feature_cols if c not in var0]

# 표준화
Z = StandardScaler().fit_transform(X_base[cols_used].values.astype(float))

# NOTEARS(linear) 실행
from notears.linear import notears_linear
NOTEARS_L1 = 0.10     # sparsity 제어(높을수록 간선↓)
NOTEARS_TH = 1e-3     # 간선 임계치

B = notears_linear(Z, lambda1=NOTEARS_L1, loss_type='l2')  # B[i,j] : j->i
B = np.asarray(B, dtype=float)

# 구조 A 결정(임계치 적용)
A = (np.abs(B) > NOTEARS_TH).astype(float)

# OLS 재적합으로 W 안정화
p = B.shape[0]
W = np.zeros_like(B)
for i in range(p):
    parents = np.where(A[i, :] == 1.0)[0]
    if parents.size == 0:
        continue
    yi = Z[:, i]
    Xi = Z[:, parents]
    coef, *_ = np.linalg.lstsq(Xi, yi, rcond=None)
    W[i, parents] = coef

Kmax = int(np.sum(A))
print("edges:", Kmax)

edges: 9


In [15]:
# k=1..Kmax 스윕, 모델 학습/저장, 최적 결과
models = make_models()
long_rows=[]; wide_rows=[]

for k in range(1, Kmax+1):
    Xm   = build_feature_df(X_base[cols_used], cols_used, W, "mul",  k)
    Xmw  = build_feature_df(X_base[cols_used], cols_used, W, "mulw", k)
    Xmmw = pd.concat([Xm, Xmw], axis=1)

    feature_sets=[("original", X_base), ("m", Xm), ("mw", Xmw), ("m+mw", Xmmw)]
    for mname, model in models:
        for fname, Xf in feature_sets:
            sets = [("O", X_base)] if fname=="original" else [("F", Xf), ("OF", pd.concat([X_base, Xf], axis=1))]
            for sname, Xset in sets:
                t0=time.time()
                clf = (LGBMClassifier(**LGBM_PARAMS) if (_HAS_LGBM and mname=="LightGBM") else model)
                metrics=run_cv(clf, Xset, y, n_splits=N_SPLITS, seed=RANDOM_STATE)
                secs=time.time()-t0
                if VERBOSE_PRINT: print(one_line(DAG_NAME,k,mname,sname,metrics,secs))
                append_log_csv(dict(dag=DAG_NAME,k=k,model=mname,set=sname,feature=fname,**metrics,seconds=round(secs,3)))
                long_rows.append(dict(DAG=DAG_NAME,k=k,Model=mname,Feature=fname,Set=sname,**metrics))
                wide_rows.append(dict(dag=DAG_NAME,k=k,model=mname,feature=fname,set=sname,**metrics))

# 리포트 저장
df_long=pd.DataFrame(long_rows)
df_long.to_csv("./reports/report_long.csv", index=False, encoding="utf-8-sig")
pv=df_long.pivot_table(index=["DAG","k","Model","Feature"], columns="Set",
                       values=["f1","AUPRC","AUROC","Brier","ECE"], aggfunc="mean")
pv.to_csv("./reports/report_pivot.csv", encoding="utf-8-sig")
save_user_style_wide_csv(wide_rows, Path("./reports/report_all_like_user.csv"))
print("saved reports")

# 모델별 최적(우선순위: AUROC desc, AUPRC desc, f1 desc, Brier asc, ECE asc)
order_cols=["AUROC","AUPRC","f1","Brier","ECE"]
ascending =[False,   False,   False, True,   True]
best_df=(df_long.sort_values(order_cols, ascending=ascending)
               .groupby("Model", as_index=False).head(1).reset_index(drop=True))
best_df.to_csv("./reports/best_per_model.csv", index=False, encoding="utf-8-sig")
best_df


DAG=NOTEARS | k=1 | model=Logit | set=O | f1=0.1164, AUPRC=0.1302, AUROC=0.8490, Brier=0.1720, ECE=0.3346 | time=0.65s
DAG=NOTEARS | k=1 | model=Logit | set=F | f1=0.0590, AUPRC=0.0376, AUROC=0.6500, Brier=0.2368, ECE=0.4470 | time=0.24s
DAG=NOTEARS | k=1 | model=Logit | set=OF | f1=0.1162, AUPRC=0.1359, AUROC=0.8509, Brier=0.1719, ECE=0.3342 | time=0.79s
DAG=NOTEARS | k=1 | model=Logit | set=F | f1=0.0590, AUPRC=0.0376, AUROC=0.6500, Brier=0.2367, ECE=0.4475 | time=0.23s
DAG=NOTEARS | k=1 | model=Logit | set=OF | f1=0.1164, AUPRC=0.1344, AUROC=0.8508, Brier=0.1717, ECE=0.3340 | time=0.83s
DAG=NOTEARS | k=1 | model=Logit | set=F | f1=0.0589, AUPRC=0.0376, AUROC=0.6500, Brier=0.2369, ECE=0.4460 | time=0.24s
DAG=NOTEARS | k=1 | model=Logit | set=OF | f1=0.1155, AUPRC=0.1373, AUROC=0.8513, Brier=0.1719, ECE=0.3339 | time=0.63s
DAG=NOTEARS | k=1 | model=RandomForest | set=O | f1=0.3142, AUPRC=0.4542, AUROC=0.9112, Brier=0.0159, ECE=0.0054 | time=27.08s
DAG=NOTEARS | k=1 | model=RandomFores

,DAG,k,Model,Feature,Set,f1,AUPRC,AUROC,Brier,ECE
0,NOTEARS,6,XGBoost,m,OF,0.480466,0.520076,0.929076,0.014514,0.009314
1,NOTEARS,4,RandomForest,mw,OF,0.317230,0.470836,0.921899,0.015581,0.005860
2,NOTEARS,2,LightGBM,m,OF,0.490945,0.484822,0.917514,0.017096,0.013422
3,NOTEARS,2,Logit,m+mw,OF,0.115536,0.138396,0.851684,0.171776,0.333832
